## CRF model

Modelo CRF para ato de aposentadoria (com dados rotulados manualmente)

In [63]:
# !pip install sklearn_crfsuite
# !pip install nltk
import pandas as pd
import sklearn_crfsuite
import nltk
from nltk.tokenize import word_tokenize


In [64]:
tokens = '/home/thais/Documentos/Knedle/experiments/members/jose_reinaldo/TCDF_aposentadoria/raw_tokenized_atos.parquet'
texts = '/home/thais/Documentos/Knedle/experiments/members/jose_reinaldo/TCDF_aposentadoria/labelbox_texts_atos.parquet'
# tokens tratados
texts = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet"
tokens = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet"

df1 = pd.read_parquet(tokens)
df2 = pd.read_parquet(texts)

# df2.groupby('ato').count()
df2["ato"].value_counts()

# result = pd.merge(df1, df2, on=["arquivo_rast"])
# result.to_csv('result.csv')
# result.head()

aviso_de_aditamento_contratual              463
extrato_de_contrato_ou_convenio             442
aviso_de_licitacao                          235
aviso_de_revogacao_anulacao_de_licitacao     16
aviso_de_suspensao_de_licitacao              12
Name: ato, dtype: int64

In [65]:
df= pd.read_csv('result.csv')
result = pd.DataFrame(df)
result.tokenized_text = result.tokenized_text.apply(lambda x: x.encode('ascii','ignore'))

Load csv with text and labels

In [66]:
x = []
y = []
for row in range(len(result)):
    if pd.notna(result['treated_text'][row]):
        text_iob = result['tokenized_text'][row].decode("utf-8").replace("', '", '').replace("[' ", '').replace("']", '')
        x.append(word_tokenize(result['treated_text'][row]))
        y.append(text_iob.split())
len(x), len(y)

(1168, 1168)

In [67]:
# Tirando os rotulos que nao batem com os dados
continua = True
while continua == True:
    continua = False
    for i in range(len(x)):
        if len(x[i]) > len(y[i]):
            x[i].pop()
            continua = True

        if len(x[i]) < len(y[i]):
            y[i].pop()
            continua = True

Create dictionary feature for each word in each sequence in x

In [68]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': '' if i == 0 else sentence[i-1].lower(),
                'word_after:': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

In [69]:
x[2]

[{'word': 'extrato',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': '',
  'word_after:': 'de',
  'BOS': True,
  'EOS': False},
 {'word': 'de',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'extrato',
  'word_after:': 'contrato',
  'BOS': False,
  'EOS': False},
 {'word': 'contrato',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'de',
  'word_after:': '1',
  'BOS': False,
  'EOS': False},
 {'word': '1',
  'capital_letter': False,
  'all_capital': False,
  'isdigit': True,
  'word_before': 'contrato',
  'word_after:': ')',
  'BOS': False,
  'EOS': False},
 {'word': ')',
  'capital_letter': False,
  'all_capital': False,
  'isdigit': False,
  'word_before': '1',
  'word_after:': 'contrato',
  'BOS': False,
  'EOS': False},
 {'word': 'contrato',
  'capital_letter': True,
  'all_capital': False,
  'isdigit': False,
  'word_before': ')',
  'word_after:': 'n.º',
  'BOS': Fals

Separate train and test splits (in order)

In [70]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(y_train)


(934, 234, 934)

In [71]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [72]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

0.6733342759591998

In [73]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))

                            precision    recall  f1-score   support

              B-NUM_AJUSTE      0.333     0.017     0.033       116
                B-PROCESSO      1.000     0.023     0.045       130
             B-CONTRATANTE      0.831     0.738     0.781       206
              B-CONTRATADA      0.526     0.174     0.261       115
              I-CONTRATADA      0.528     0.213     0.303       536
              B-OBJ_AJUSTE      0.667     0.483     0.560       116
              I-OBJ_AJUSTE      0.521     0.707     0.600      3781
                   B-VALOR      0.214     0.045     0.075        66
                      B-PT      0.333     0.053     0.091        57
                      I-PT      0.000     0.000     0.000         4
                B-VIGENCIA      0.625     0.491     0.550       112
                I-VIGENCIA      0.758     0.856     0.804      1413
             I-CONTRATANTE      0.939     0.853     0.894       927
         B-DATA_ASSINATURA      0.167     0.018